In [3]:
from knn import kNN
from load_mini_imagenet import load_mini_imagenet_data, split_for_n_shot

In [4]:
X_test, y_test = load_mini_imagenet_data("mini-imagenet-cache-test.pkl")

In [5]:
X_support, y_support, X_test, y_test = split_for_n_shot(X_test, y_test, n=5)

In [6]:
# extract pixels as features
test_features = X_test.reshape(X_test.shape[0], -1)
support_features = X_support.reshape(X_support.shape[0], -1)
print(test_features.shape)
print(support_features.shape)

(11900, 21168)
(100, 21168)


In [11]:
correct = 0
for feat, label in tqdm(zip(test_features, y_test)):
    predicted = kNN(support_features, y_support, feat, k=3)
    if predicted == label:
        correct += 1

11900it [01:07, 175.30it/s]


In [10]:
# k = 1, euclidean
print(correct / len(test_features))

0.09462184873949579


In [12]:
# k = 3, euclidean
print(correct / len(test_features))

0.06495798319327731
